In [2]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.neural_network import MLPRegressor

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g
import src.eval_utils as e_u

c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DungeonMaster3000\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


В файле baseline.ipynb было показано, что использование эмбеддинга для сообщения коммита позволяет увеличить точность моделей. Попробуем также tfidf с уменьшением размерности, а также предобученный мультиязыковой кодировщик предложений на основе Transformer и CNN

Загрузим датасет и предобработаем категориальные фичи

In [3]:
df = d_u.get_preprocess_data()

In [4]:
df.sample(3)

,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,Bob,...,Eve,Mallory,Peggy,Trudy,Victor,Wendy,no_work_d,work_d,no_work_h,work_h
317,added trapconductor,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
300,remove logstash and change data path in filebeat,5,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1
95,Update src/express.py,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [5]:
X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

Сначала сразу посмотрим результат модели с эмбеддингами сообщений, которые мы получаем с помощью предобученной мультиязыковой модели

In [6]:
msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_idxs, test_idxs = X_train.index, X_test.index

In [8]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [9]:
X_train = np.concatenate((X_train, msg_embs[train_idxs]), axis=1)
X_test = np.concatenate((X_test, msg_embs[test_idxs]), axis=1)

In [10]:
zoo_models = [
    LinearRegression(),
    SGDRegressor(random_state=42),
    Ridge(random_state=42),
    Lasso(random_state=42),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(),
    MLPRegressor(random_state=42, max_iter=100000)
    ]

for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднее квадратное отклонение: 3.9255156937438884

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 1.5442650283714352

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 1.3328333886580885

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 0.86288125

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 0.7879865467367375

Модель - SVR()
Среднее квадратное отклонение: 2.155657021426432

Модель - MLPRegressor(max_iter=100000, random_state=42)
Среднее квадратное отклонение: 1.7726122043258286



Сразу заметно улучшение качества при использовании предобученной модели для генерации эмбеддинга предложения по сравнению с tfidf на том же трэине и тесте(см. baseline.ipynb)

Посмотрим среднеквадратическое отклонение на кросс-валидации

In [11]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
ss = StandardScaler()
X_eval = ss.fit_transform(X)
X_eval = np.concatenate((X_eval, msg_embs), axis=1)

for model in zoo_models:
    print(f"Модель - {model}")
    mse_mean = -np.mean(cross_val_score(model, X_eval, y, cv=cv, scoring="neg_mean_squared_error"))
    print(f"Среднеквадратическое отклонение: {mse_mean}")
    # preds = model.predict(X_test)
    # print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднеквадратическое отклонение: 3.6631705664438847

Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 1.5791955326983218

Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 1.3792463938249164

Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 1.7560374454365077

Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 1.470161025026534

Модель - SVR()
Среднеквадратическое отклонение: 2.7170945885316122

Модель - MLPRegressor(max_iter=100000, random_state=42)
Среднеквадратическое отклонение: 1.4481975235824769



В отличие от фиксированного трэина и теста на кросс-валидации себя лучше всего показала линейная модель с L2 регуляризацией и однослойная нейронная сеть

Оценим, как понижение размерности эмбеддинга и применение StandardScaler влияет на качество 

Сначала посмотрим на GradientBoostingRegressor

In [12]:
model = GradientBoostingRegressor(random_state=42)

In [13]:
print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print("На оригинальных эмбеддингах + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, scale=True)

На оригинальных эмбеддингах
-1 - 1.470161025026534
На оригинальных эмбеддингах + масштабирование
-1 - 1.470161025026534


In [14]:
print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

С использованием PCA
5 - 1.5148551894647695
10 - 1.4560367799511498
15 - 1.4877978299274321
20 - 1.5214871692524619
25 - 1.482816085116543
50 - 1.5609884075840905
100 - 1.8245619626528966
200 - 1.82374322613191
-1 - 1.470161025026534


In [15]:
print("С использованием PCA + масштабироавние")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca", scale=True)

С использованием PCA + масштабироавние
5 - 1.489295710440322
10 - 1.4538726678656113
15 - 1.4751021555784114
20 - 1.5180275216073238
25 - 1.478111264499026
50 - 1.566128901800436
100 - 1.7434172615126369
200 - 1.817320826978331
-1 - 1.470161025026534


In [16]:
print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

print()

print("С использованием SVD + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

С использованием SVD
5 - 1.555566258795838
10 - 1.5138571113038022
15 - 1.4582576725560403
20 - 1.5126696828208244
25 - 1.512433820633022
50 - 1.590714880944181
100 - 1.8915103406943867
200 - 1.982722487377621
-1 - 1.470161025026534

С использованием SVD + масштабирование
5 - 1.530622570109774
10 - 1.49362152904437
15 - 1.46066956586588
20 - 1.4773792398921621
25 - 1.5572477635659516
50 - 1.694677194110767
100 - 1.8430409305398765
200 - 1.9055306870088375
-1 - 1.470161025026534


Для базового градиентного бустинга лучший результат - 1.435900249421395 при понижении размерности эмбеддинга с помощью TruncatedSVD до 15 без применения масштабирования. Однако если оставлять исходный размер эмбеддинга, то имеем mse - 1.470161025026534.

Теперь протестируем понижение размерности с моделью Ridge

In [17]:
model = Ridge(random_state=42)

print("Ridge:\n")

print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print()

print("На оригинальных эмбеддингах + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, scale=True)

print()

print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

print()

print("С использованием PCA + масштабироавние")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca", scale=True)

print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

print()

print("С использованием SVD + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

Ridge:

На оригинальных эмбеддингах
-1 - 1.3672444447352596

На оригинальных эмбеддингах + масштабирование
-1 - 1.3791697289464646

С использованием PCA
5 - 1.608985540104505
10 - 1.5056748797877482
15 - 1.4955045541755039
20 - 1.503601474094973
25 - 1.5125714235983743
50 - 1.4040745553935343
100 - 1.3861526819052499
200 - 1.3695839856638639
-1 - 1.3672444447352596

С использованием PCA + масштабироавние
5 - 1.6133605738392085
10 - 1.5089069094553893
15 - 1.4981613178766167
20 - 1.5100405878337801
25 - 1.5293546334825607
50 - 1.4149903442395846
100 - 1.3955291324888093
200 - 1.3803108606792143
-1 - 1.3791697289464646
С использованием SVD
5 - 1.715263243857483
10 - 1.4772050738427398
15 - 1.4645535959873204
20 - 1.470583933977404
25 - 1.4788954132985617
50 - 1.4069166640223916
100 - 1.3829728648305806
200 - 1.370333452797362
-1 - 1.3672444447352596

С использованием SVD + масштабирование
5 - 1.7145391888961394
10 - 1.4737637095027007
15 - 1.4610747812439862
20 - 1.4712964425881492
25 - 

Лучшая оценка mse - 1.3672444447352596. Она получается на неуменьшенных эмбеддингах и без масштабирования.

Теперь посмотрим на нейронную сеть

In [18]:
# max_iter = 10000, чтобы модель могла сойтись
model = MLPRegressor(random_state=42, max_iter=10000)

In [19]:
print("MLP:\n")

print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print()

print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

print()

print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")


MLP:

На оригинальных эмбеддингах
-1 - 1.4306585077994984

С использованием PCA
5 - 1.5748183994232874
10 - 1.5974804688051203
15 - 1.7617100799757708
20 - 1.4290993655345103
25 - 1.2983291254581564
50 - 1.2237945874117258
100 - 1.2416440619267823
200 - 1.6417422891675344
-1 - 1.4306585077994984

С использованием SVD
5 - 1.7472114611321974
10 - 1.4974275998975868
15 - 1.3806399853749596
20 - 1.3528996280903498
25 - 1.2719689577502364
50 - 1.297518674064174
100 - 1.2809035338071357
200 - 1.7584703650445874
-1 - 1.4306585077994984


На данный момент нейронная сеть с понижением размерности эмбеддинга с помощью PCA до 50 показала наилучшую оценку mse - 1.2237945874117258

Попробуем немного изменить архитектуру

In [20]:
model = MLPRegressor(hidden_layer_sizes=(300, 150, 25,), random_state=42, max_iter=100000)
e_u.eval_emb_reduction(model, X, y, msg_embs)

-1 - 1.218504096837509


In [21]:
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

5 - 1.7855156634848783
10 - 1.3843189143725183
15 - 1.374109487581828
20 - 1.3859449720839223
25 - 1.3885308917918893
50 - 1.1908885605419521
100 - 1.2892984050555674
200 - 1.3879867785325806
-1 - 1.218504096837509


При использовании PCA сновой архитектурой мы немног улучшили mse, но не оч значительно.

Ячейки выше показывает, что при понижении размерности можно достичь примерн такого же качества, что и при эмбеддингеоригинального размера, но проще ничего не трогать и получать аналогичное качество работы модели или даже выше. Также StandardScaler можно не применять, при его использовании среднеквадратическое отклонение на кросс-валидации слегка увеличивается, как на бустинге, так и на линейной модели с регуляризацией.

Лучше всего себя показала нейронная сеть с понижением размера эмбеддинга. Также можно не понижать размерность эмбеддинга сообщения коммита, но в таком случае необходимо увеличивать количество слоев. Следующими по качеству предсказания идут Ridge и GradientBoostingRegressor.

Отдельно проверять tfidf в данный момент уже не буду, так как эмбеддинги предложений получаемые с помощью предобученной модели сразу показали гораздо лучший результат.